In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [2]:
kart_data = pd.read_csv("../Data/data.csv",sep=';',decimal=',')

/home/paulo/PycharmProjects/kart-scraping-analysis/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (16,18,19,22,25,28,29,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Select only rental races

In [3]:
clean_data = kart_data.loc[kart_data['Classe']=='RENTAL']

In [4]:
def time_spliter(t):
    keys = ['milliseconds','seconds','minutes']
    if isinstance(t, str):
        values = reversed([int(x) for x in re.split("[:.]+", t)])
        time_dict = dict(zip(keys, values))
        return pd.Timedelta(**time_dict)
    else:
        return pd.Timedelta('nat')

In [5]:
columns = ['Melhor Tempo', 'Total Tempo', '2a melhor']
for column in columns:
    clean_data.loc[:,(column)] = clean_data[column].apply(time_spliter)
clean_data = clean_data.loc[clean_data['Melhor Tempo'].notna()]

In [6]:
clean_data.loc[:,('Dia')] = pd.to_datetime(clean_data['Dia'] + '/2019', format='%d/%m/%Y')
clean_data['Week'] = clean_data['Dia'].dt.strftime('%U')
clean_data.loc[:,('No.')] = clean_data['No.'].apply(lambda x: str(int(x)))

In [7]:
best_lap = clean_data.groupby('uid')['Melhor Tempo'].min()\
                .reset_index().rename({'Melhor Tempo':'Best Circuit Lap'}, axis=1)
clean_data = clean_data.merge(best_lap,how='left',on='uid')
clean_data['Dif from best'] = clean_data['Melhor Tempo'] - clean_data['Best Circuit Lap']

In [24]:
group_data = clean_data.groupby(['Week','No.'])['Best Circuit Lap'].mean(numeric_only=False).reset_index()

In [25]:
group_data.loc[group_data['No.']=='10']

,Week,No.,Best Circuit Lap
1,17,10,0 days 00:00:53.568222222
53,18,10,0 days 00:00:48.702541666
133,19,10,0 days 00:00:57.292071428
187,20,10,0 days 00:00:56.854023255
258,21,10,0 days 00:00:55.011625
342,22,10,0 days 00:00:46.976261904
416,23,10,0 days 00:00:45.852939393
492,24,10,0 days 00:00:46.692000
549,25,10,0 days 00:00:51.593263157
632,26,10,0 days 00:00:58.421228571
